# 01 - Load data into an Azure SQL non-partitioned table

The sample is using the new sql-spark-connector (https://github.com/microsoft/sql-spark-connector). Make sure you have installed it before running this notebook. Maven Coordinates: `com.microsoft.azure:spark-mssql-connector:1.0.0`

## Notes on terminology

The term "row-store" is used to identify and index that is not using to store its data.

## Samples

In this notebook there are three samples

- Load data into a table without indexes
- Load data into a table with row-store indexes
- Load data into a table with columns-store indexes


## Setup

Define variables used thoughout the script. Azure Key Value has been used to securely store sensitive data. More info here: [Create an Azure Key Vault-backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope)

In [4]:
val scope = "key-vault-secrets"

val storageAccount = "dmstore2";
val storageKey = dbutils.secrets.get(scope, "dmstore2-2");

val server = dbutils.secrets.get(scope, "srv001").concat(".database.windows.net");
val database = "ApacheSpark";
val user = dbutils.secrets.get(scope, "dbuser001");
val password = dbutils.secrets.get(scope, "dbpwd001");
val table = "dbo.LINEITEM_LOADTEST"

val url = s"jdbc:sqlserver://$server;databaseName=$database;"

scope: String = key-vault-secrets
storageAccount: String = dmstore2
storageKey: String = [REDACTED]
server: String = [REDACTED].database.windows.net
database: String = ApacheSpark
user: String = [REDACTED]
password: String = [REDACTED]
table: String = dbo.LINEITEM_LOADTEST
url: String = jdbc:sqlserver://[REDACTED].database.windows.net;databaseName=ApacheSpark;

Configure Spark to access Azure Blob Store

In [6]:
spark.conf.set(s"fs.azure.account.key.$storageAccount.blob.core.windows.net", storageKey);

Load the Parquet file generated in `00-create-parquet-file` notebook that contains LINEITEM data partitioned by Year and Month

In [8]:
val li = spark.read.parquet(s"wasbs://tpch@$storageAccount.blob.core.windows.net/10GB/parquet/lineitem")

li: org.apache.spark.sql.DataFrame = [L_ORDERKEY: int, L_PARTKEY: int ... 15 more fields]

Loaded data is split in 20 dataframe partitions

In [10]:
li.rdd.getNumPartitions

res2: Int = 20

Show schema of loaded data

In [12]:
li.printSchema

root
-- L_ORDERKEY: integer (nullable = true)
-- L_PARTKEY: integer (nullable = true)
-- L_SUPPKEY: integer (nullable = true)
-- L_LINENUMBER: integer (nullable = true)
-- L_QUANTITY: decimal(15,2) (nullable = true)
-- L_EXTENDEDPRICE: decimal(15,2) (nullable = true)
-- L_DISCOUNT: decimal(15,2) (nullable = true)
-- L_TAX: decimal(15,2) (nullable = true)
-- L_RETURNFLAG: string (nullable = true)
-- L_LINESTATUS: string (nullable = true)
-- L_SHIPDATE: date (nullable = true)
-- L_COMMITDATE: date (nullable = true)
-- L_RECEIPTDATE: date (nullable = true)
-- L_SHIPINSTRUCT: string (nullable = true)
-- L_SHIPMODE: string (nullable = true)
-- L_COMMENT: string (nullable = true)
-- L_PARTITION_KEY: integer (nullable = true)

Now, make sure you create on your Azure SQL the following LINEITEM table:
```sql
create table [dbo].[LINEITEM_LOADTEST]
(
	[L_ORDERKEY] [int] not null,
	[L_PARTKEY] [int] not null,
	[L_SUPPKEY] [int] not null,
	[L_LINENUMBER] [int] not null,
	[L_QUANTITY] [decimal](15, 2) not null,
	[L_EXTENDEDPRICE] [decimal](15, 2) not null,
	[L_DISCOUNT] [decimal](15, 2) not null,
	[L_TAX] [decimal](15, 2) not null,
	[L_RETURNFLAG] [char](1) not null,
	[L_LINESTATUS] [char](1) not null,
	[L_SHIPDATE] [date] not null,
	[L_COMMITDATE] [date] not null,
	[L_RECEIPTDATE] [date] not null,
	[L_SHIPINSTRUCT] [char](25) not null,
	[L_SHIPMODE] [char](10) not null,
	[L_COMMENT] [varchar](44) not null,
	[L_PARTITION_KEY] [int] not null
) 
```

## Load data into a table with no indexes

In Azure SQL terminology an Heap is a table with no clustered index. In this sample we'll load data into a table that as no index (clustered or non-clustered) as is not partitioned. This is the simplest scenario possibile and allows parallel load of data.

### Note:
Parallel load *cannot* happen if you have row-store indexes on the table. If you want to bulk load data in parallel into a table that has row-store indexes, you must use partitioning. If you are planning to add indexes to your table, and data to be loaded in the table is in the terabyte range, you want to use partitioing and have indexes created before bulk loading data into Azure SQL, as otherwise creating index once the table is already loaded will use a significat amout of resources.

To enable parallel load the option `tableLock` must be set to `true`. This will prevent any other access to the table, other then the one done for performing the bulk load operations.

In [16]:
li.write
  .format("com.microsoft.sqlserver.jdbc.spark") 
  .mode("overwrite")   
  .option("truncate", "true") 
  .option("url", url) 
  .option("dbtable", "dbo.LINEITEM_LOADTEST") 
  .option("user", user) 
  .option("password", password) 
  .option("reliabilityLevel", "BEST_EFFORT") 
  .option("tableLock", "true") 
  .option("batchsize", "100000")
  .option("schemaCheckEnabled", "false") // needed to avoid clash of NULLable columns vs NON-NULLable colums
  .save()

## Load data into a table with row-store indexes

If table is not partitioned, there are no options to bulk load data in parallel into the desired table. The only way to avoid locking and deadlocks is to load everything by serializing the bulk load operations. As you can expect, performance won't be the optimal.

Create the following index on the table
```sql
create clustered index IXC on dbo.[LINEITEM_LOADTEST] ([L_COMMITDATE]);

create unique nonclustered index IX1 on dbo.[LINEITEM_LOADTEST] ([L_ORDERKEY], [L_LINENUMBER]);

create nonclustered index IX2 on dbo.[LINEITEM_LOADTEST] ([L_PARTKEY]); 
```

Load data by coalescing all dataframe partitions into just one

In [20]:
val url = s"jdbc:sqlserver://$server;databaseName=$database;"

li.coalesce(1)
  .write
  .format("com.microsoft.sqlserver.jdbc.spark") 
  .mode("overwrite")   
  .option("truncate", "true") 
  .option("url", url) 
  .option("dbtable", "dbo.LINEITEM_LOADTEST") 
  .option("user", user) 
  .option("password", password) 
  .option("reliabilityLevel", "BEST_EFFORT") 
  .option("tableLock", "false") 
  .option("batchsize", "100000")
  .option("schemaCheckEnabled", "false")
  .save()

## Load data into a table with (only) column-store indexes

If a table has only column-store indexes, data load can happen in parallel, as there is no sorting needed.

Empty table if needed, to speed up index deletion

```sql
truncate table dbo.[LINEITEM_LOADTEST];
```

Drop the previously create indexes if needed:
```sql
drop index IXC on dbo.[LINEITEM_LOADTEST];
drop index IX1 on dbo.[LINEITEM_LOADTEST];
drop index IX2 on dbo.[LINEITEM_LOADTEST];
```

And the create a clustered columnstore index:

```sql
create clustered columnstore index IXCCS on dbo.[LINEITEM_LOADTEST]
```

By loading 1048576 rows at time, to land directly into a compressed segment. `tableLock` options must be set to `false` to avoid table lock that will prevent parallel load. Data with be loaded in parallel, using as many as Apache Spark workers are available.

In [24]:
li.write 
  .format("com.microsoft.sqlserver.jdbc.spark") 
  .mode("overwrite")   
  .option("truncate", "true") 
  .option("url", url) 
  .option("dbtable", "dbo.LINEITEM_LOADTEST") 
  .option("user", user) 
  .option("password", password) 
  .option("reliabilityLevel", "BEST_EFFORT") 
  .option("tableLock", "false") 
  .option("batchsize", "1048576")   
  .option("schemaCheckEnabled", "false")
  .save()